In [ ]:
# pip install -r ./requirement.txt -q
!pip uninstall torch

In [ ]:
!pip install torch --extra-index-url https://download.pytorch.org/whl/cu117

In [2]:
# Imports
from chromadb.config import Settings
from urllib.error import HTTPError
from dataclasses import replace
from dotenv import load_dotenv
from tqdm import tqdm
import numpy as np
import tiktoken # OpenAI's open-source tokenizer
import chromadb
import logging
import random # to sample multiple elements from a list
import arxiv
import time
import os # operating system dependent functionality, to walk through directories and files

from langchain.text_splitter import RecursiveCharacterTextSplitter # recursively tries to split by different characters to find one that works
from langchain.document_loaders import PyPDFDirectoryLoader # loads pdfs from a given directory
from langchain.chains import ConversationalRetrievalChain # looks up relevant documents from the retriever per history and question.
from langchain.text_splitter import CharacterTextSplitter # splits the content
from langchain.embeddings import HuggingFaceBgeEmbeddings # wrapper for HuggingFaceBgeEmbeddings models
from langchain.llms import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain
from langchain.document_loaders import ArxivLoader # loads paper for a given id from Arxiv
from langchain.document_loaders import PyPDFLoader # loads a given pdf
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import TextLoader # loads a given text
from langchain.retrievers import ArxivRetriever # loads relevant papers for a given paper id from Arxiv
from chromadb.utils import embedding_functions # loads Chroma's embedding functions from OpenAI, HuggingFace, SentenceTransformer and others
from langchain.chat_models import ChatOpenAI # wrapper around OpenAI LLMs
from langchain.vectorstores import Chroma # wrapper around ChromaDB embeddings platform
from langchain.chains import RetrievalQA
from langchain import HuggingFaceHub # wrapper around HuggingFaceHub models

from transformers import AutoTokenizer, pipeline, logging
from auto_gptq import AutoGPTQForCausalLM, BaseQuantizeConfig

load_dotenv() # loads env variables

CUDA extension not installed.
CUDA extension not installed.


True

In [3]:
# !mkdir arxiv_papers
# dirpath = "arxiv_papers"

# search = arxiv.Search(
#   query = "2303.18223" # ID of the paper A Survey of Large Language Models
# )

# for result in tqdm(search.results()):
#     result.download_pdf(dirpath=dirpath)
#     print(f"-> Paper id {result.get_short_id()} with title '{result.title}' is downloaded.")

In [4]:
# papers = []
# loader = DirectoryLoader('./arxiv_papers/', glob="./*.pdf", loader_cls=PyPDFLoader)
# papers = loader.load()
# print("Total number of pages loaded: ", len(papers)) # Total number of pages loaded:  85

Total number of pages loaded:  124


In [7]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50
)
with open('churchill.txt') as f:
    churchill_speech = f.read()

paper_chunks = text_splitter.split_documents(churchill_speech)
paper_chunks =text_splitter.create_documents([churchill_speech])
len(paper_chunks)

47

In [ ]:
# veifying the average length of the chunks
chunk_lengths = [len(paper_chunk.page_content) for paper_chunk in paper_chunks]
np.average(chunk_lengths)


# Embedding wih BG2

In [8]:
# Downloading HuggingFace BG embeddings
model_name = "BAAI/bge-base-en"
encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity

embedding_function = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    # model_kwargs={'device': 'cuda'},
    encode_kwargs=encode_kwargs
)

In [ ]:
# !python -c "import torch; print(torch.cuda.is_available())"

In [9]:
persist_directory="./chromadb/"

vectordb = Chroma.from_documents(
    documents=paper_chunks, # text data that you want to embed and store
    embedding=embedding_function, # used to convert the documents into embeddings
    persist_directory=persist_directory, # tells Chroma where to store its data
    collection_name="arxiv_papers" #  gives a name to the collection of embeddings, which will be helpful for retrieving specific groups of embeddings later.
)

In [11]:
vectordb

AttributeError: 'Chroma' object has no attribute 'size'